In [34]:
import sys 
import recsys
import recsys.algorithm
from recsys.algorithm.factorize import SVD
from recsys.algorithm.factorize import SVDNeighbourhood
from recsys.datamodel.data import Data
from recsys.evaluation.prediction import RMSE,MAE
import numpy as np
import pandas as pd
recsys.algorithm.VERBOSE = True
movies_file = "/media/sourabhkondapaka/Sourabh's/main_project/sandbox/ml-latest-small/movies.csv"
movies = pd.read_table(movies_file, sep=',')
movies.columns = ['movie_id','title','genres']
ratings_file = "/home/sourabhkondapaka/Desktop/ratingsss.csv"

movies = movies
K = 100
PERCENT_TRAIN = 85
#self.users = users
#Need to provide a default file location for ratings.csv instead of loading everytime.run below 2lines only once 
#or just provide this file instead.
#self.users.to_csv("/home/sourabhkondapaka/Desktop/ratingsss.csv",index= False)
ratings_file = ratings_file  #Give your path to ratings.csv created from above 2 lines.
data = None
svd = None
recommend_movies_list = None
recommend_movies_ids = None
similar_movies_list = None
similar_movies_ids = None        
movie_id = None
train = None
test = None 






'''def initiate(ratings_file,movies):#No need to pass as ,will be provided in views.py
    #self.users = users
    #Need to provide a default file location for ratings.csv instead of loading everytime.run below 2lines only once 
    #or just provide this file instead.
    #self.users.to_csv("/home/sourabhkondapaka/Desktop/ratingsss.csv",index= False)
    ratings_file = ratings_file  #Give your path to ratings.csv created from above 2 lines.
    data = None
    svd = None
    recommend_movies_list = None
    recommend_movies_ids = None
    similar_movies_list = None
    similar_movies_ids = None        
    movie_id = None
    train = None
    test = None '''




def compute_svd():      
    global data
    global train
    global  test
    data = Data()
    global PERCENT_TRAIN
    data.load(ratings_file, sep=',', format={'col':0, 'row':1 ,'value':2, 'ids':float})
    train , test = data.split_train_test(percent=PERCENT_TRAIN)    
    global svd 
    svd = SVDNeighbourhood()
    svd.set_data(train)    
    svd.compute(k=K, min_values=1, pre_normalize=None, mean_center=False, post_normalize=True)

def similarity_measure(movie1 , movie2): #gives a similarity measure value between -1 to 1
    global svd
    return round(svd.similarity(movie1,movie2),4)

def recommend_movies(user_id):
    global svd
    l = svd.recommend(user_id, n=10, only_unknowns=True, is_row=False)
    global recommend_movies_list
    recommend_movies_list = []
    global recommend_movies_ids
    recommend_movies_ids = []
    for p in l:
        #movie names
        bb = str(movies.ix[movies['movie_id'] == p[0] ]['title']).split()    
        q = bb.index('Name:')
        bb = ' '.join(bb[1:q])
        recommend_movies_list.append(bb) 
        #movie ids
        gg = movies.ix[movies['movie_id'] == p[0]]
        gg = gg.reset_index()
        del gg['index']
        gg = gg.ix[:,0:2].as_matrix(columns = None).tolist()
        recommend_movies_ids.append(gg[0][0])
    return recommend_movies_list,recommend_movies_ids

def get_similar_movies(movie1):#Returns a PYTHON list for similar movies.
    #movie1 = int(movie1)
    global svd
    l = svd.similar(movie1)   
    global similar_movies_list
    similar_movies_list = []
    global similar_movies_ids
    similar_movies_ids = []
    l = l[1:]

    for p in l:
        #getting movie names
        bb = str(movies.ix[movies['movie_id'] == p[0] ]['title']).split()    
        q = bb.index('Name:')
        bb = ' '.join(bb[1:q])
        similar_movies_list.append(bb)   
        #getting movie id's
        similar_movies_ids.append(p[0])

    return similar_movies_list,similar_movies_ids


In [27]:
initiate(ratings_file,movies)

In [35]:
compute_svd()

In [78]:
import sys 
import recsys
import recsys.algorithm
from recsys.algorithm.factorize import SVD
from recsys.algorithm.factorize import SVDNeighbourhood
from recsys.datamodel.data import Data
from recsys.evaluation.prediction import RMSE,MAE
import numpy as np
import pandas as pd
recsys.algorithm.VERBOSE = True
movies_file = "/media/sourabhkondapaka/Sourabh's/main_project/sandbox/ml-latest-small/movies.csv"
movies = pd.read_table(movies_file, sep=',')
movies.columns = ['movie_id','title','genres']
ratings_file = "/home/sourabhkondapaka/Desktop/ratingsss.csv"





class Collaborative_filtering(object):
    def __init__(self,ratings_file,movies):#No need to pass as ,will be provided in views.py
        #self.users = users
        self.movies = movies
        self.K = 100
        self.PERCENT_TRAIN = 85
        #Need to provide a default file location for ratings.csv instead of loading everytime.run below 2lines only once 
        #or just provide this file instead.
        #self.users.to_csv("/home/sourabhkondapaka/Desktop/ratingsss.csv",index= False)
        self.ratings_file = ratings_file  #Give your path to ratings.csv created from above 2 lines.
        self.data = None
        self.svd = None
        self.recommend_movies_list = None
        self.recommend_movies_ids = None
        self.similar_movies_list = None
        self.similar_movies_ids = None
        
        self.movie_id = None
        self.train = None
        self.test = None

        
        
    def compute_svd(self):      
        #self.data = Data()
        #self.data.load(self.ratings_file, sep=',', format={'col':0, 'row':1 ,'value':2, 'ids':float})
        #self.train , self.test = self.data.split_train_test(percent=self.PERCENT_TRAIN)    
        #self.svd = SVDNeighbourhood()
        #self.svd.set_data(self.train) 
        self.svd = SVDNeighbourhood()
        self.svd.load_data(filename=ratings_file, sep=',', format={'col':0, 'row':1, 'value':2, 'ids': float})
        self.svd.compute(k=self.K,min_values=10,pre_normalize=None,mean_center=True,post_normalize=True,savefile='/tmp/movielens')

    def similarity_measure(self,movie1 , movie2): #gives a similarity measure value between -1 to 1
        return round(self.svd.similarity(movie1,movie2),4)
    
    def recommend_movies(self,user_id):
        l =  self.svd.recommend(user_id, n=10, only_unknowns=True, is_row=False)
        self.recommend_movies_list = []
        self.recommend_movies_ids = []
        for p in l:
            #movie names
            bb = str(movies.ix[movies['movie_id'] == p[0] ]['title']).split()    
            q = bb.index('Name:')
            bb = ' '.join(bb[1:q])
            self.recommend_movies_list.append(bb) 
            #movie ids
            gg = movies.ix[movies['movie_id'] == p[0]]
            gg = gg.reset_index()
            del gg['index']
            gg = gg.ix[:,0:2].as_matrix(columns = None).tolist()
            self.recommend_movies_ids.append(gg[0][0])
        return self.recommend_movies_list,self.recommend_movies_ids
    
    def get_similar_movies(self,movie1):#Returns a PYTHON list for similar movies.
        #movie1 = int(movie1)
        l = self.svd.similar(movie1)   
        self.similar_movies_list = []
        self.similar_movies_ids = []
        l = l[1:]
        
        for p in l:
            #getting movie names
            bb = str(movies.ix[movies['movie_id'] == p[0] ]['title']).split()    
            q = bb.index('Name:')
            bb = ' '.join(bb[1:q])
            self.similar_movies_list.append(bb)   
            #getting movie id's
            self.similar_movies_ids.append(p[0])
            
        return self.similar_movies_list,self.similar_movies_ids
    

In [79]:
cf = Collaborative_filtering(ratings_file,movies)

In [80]:
cf.compute_svd()

In [81]:
print cf.get_similar_movies(1)


(['Toy Story 2 (1999)', 'Toy Story 3 (2010)', 'Robin Hood (1973)', 'Social Network, The (2010)', "Bug's Life, A (1998)", 'Lion King, The (1994)', 'Basic Instinct (1992)', 'Aladdin (1992)', 'Dark Knight, The (2008)'], [3114, 78499, 3034, 80463, 2355, 364, 1092, 588, 58559])


In [48]:
from recsys.algorithm.factorize import SVD
svd = SVD()
svd.load_data(ratings_file,sep=',',format={'col':0, 'row':1, 'value':2, 'ids': float})

In [49]:
k = 100
svd.compute(k=k,min_values=10,pre_normalize=None,mean_center=True,post_normalize=True,savefile='/tmp/movielens')

In [63]:
#svd.similar(3)

In [75]:
import recsys.algorithm
recsys.algorithm.VERBOSE = True
from recsys.algorithm.factorize import SVDNeighbourhood
svd = SVDNeighbourhood()
svd.load_data(filename=ratings_file, sep=',', format={'col':0, 'row':1, 'value':2, 'ids': float})

In [76]:
k = 100
svd.compute(k=k, min_values=10, pre_normalize=None, mean_center=True, post_normalize=True)

In [77]:
print svd.recommend(22)

[(564, 4.7775693455198898), (46, 4.6623377026811781), (443, 4.6060085732703477), (448, 4.4917620471198658), (566, 4.4818837747559517), (622, 4.4658610216884824), (298, 4.4626769061148535), (287, 4.3641763772336732), (89, 4.3560057435944746), (446, 4.3525806743748046)]
